### ETL (Extract, Transform, Load)

- Extract is refers to pulling the source data from the original database or data source. With ETL, the data goes into a temporary staging area. With ELT it goes immediately into a data lake storage system.

- Transform is refers to the process of changing the structure of the information, so it integrates with the target data system and the rest of the data in that system.

- Load is refers to the process of depositing the information into a data storage system.

### ETL VS ELT ?

##### ```Tidak perlu bingung dengan istilahnya, ini teknik yang sama namun berbeda urutan```

- ETL kita melakukan transformasi pada database staging atau python
- ELT kita melakukan transformasi pada target database ( data warehouse )
> contohnya ELT biasanya adalah data lake

### Keuntungan ETL

> ETL provides a more secure way to perform these transformations because it transforms the data before putting it into the data warehouse. In contrast, ELT requires you to upload the sensitive data first. That causes it to show up in logs where SysAdmins can access to it

Here we will only use pandas as a package that help us to solve all ETL problem

> Why pandas ? Because it's so powerful

#### 1. EXTRACT

##### from open API

In [ ]:
import pandas as pd
import requests

pd.set_option('display.max_columns', 100)

In [ ]:
# OPEN API
response = requests.get("https://hacker-news.firebaseio.com/v0/item/16222426.json")
print(response.json())

In [ ]:
data_json = response.json()

In [ ]:
data_json

In [ ]:
df_json = pd.DataFrame(data_json, index=[0])

In [ ]:
df_json

### NOW WE ARE USING REAL CASE

- open folder Chapter-3 on JatimCamp5, then follow the instructions

In [ ]:
import sqlalchemy
# If there is no SQLachemy
# pip install SQLAlchemy

from Dependencies import credential_db

- connect to production jatimcamp5

In [ ]:
conn = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                               format(credential_db.db_jatimcamp5_username, 
                                                      credential_db.db_jatimcamp5_password, 
                                                      credential_db.db_jatimcamp5_host, 
                                                      credential_db.db_jatimcamp5_name))

conn

- check whether connect or not

In [ ]:
query_table = """show tables;"""

In [ ]:
df_data_table = pd.read_sql(query_table,conn)

In [ ]:
df_data_table

In [ ]:
query_listings = """select * from listings"""

In [ ]:
df_data_listings = pd.read_sql(query_listings, conn)

In [ ]:
df_data_listings.shape

In [ ]:
df_data_listings.head(2)

In [ ]:
df_data_listings.columns

### Try to get another data from production, then we move to next steps

#### 2. TRANSFORM

#### I want do somes tranformation on this LISTING DATA

> #### select columns that choosen, we can use drop function or just select column that we want

In [ ]:
df_data_listings.drop(['listing_url',
                       'thumbnail_url','medium_url',
                       'xl_picture_url','host_thumbnail_url',
                       'picture_url'],axis=1, inplace=True)

In [ ]:
df_data_listings.head(2)

> #### Edit column with UPPERCASE letter

In [ ]:
df_data_listings['country'] = df_data_listings['country'].apply(lambda x: x.upper())

# OR
"""
 df_data_listings['country'] = df_data_listings['country'].str.upper()
"""

df_data_listings['country'].tail(2)

> #### change price to IDR

In [ ]:
df_data_listings['price'].head(2)

In [ ]:
df_data_listings['price'] = df_data_listings['price'].apply(lambda x: x.lstrip('$'))

In [ ]:
df_data_listings['price'] = df_data_listings['price'].astype('float')

In [ ]:
df_data_listings['price'].head()

In [ ]:
df_data_listings['price_in_IDR'] = df_data_listings['price'] * 14700 # 14700 ( USD to IDR )

In [ ]:
df_data_listings['price_in_IDR'].head()

> #### change dtype date time

In [ ]:
df_data_listings['last_review'] = pd.to_datetime(df_data_listings['last_review'], errors='coerce')

In [ ]:
df_data_listings['last_review'].head()

In [ ]:
df_data_listings['last_review'] == '2016-09-05'

In [ ]:
df_data_listings[df_data_listings['last_review'] > '2016-08-01'].head()

#### I want do somes tranformation on this REVIEWS DATA

> #### find language in comment

In [ ]:
query_reviews = """select * from reviews"""

In [ ]:
df_data_reviews = pd.read_sql(query_reviews, conn)

In [ ]:
df_data_reviews.head()

In [ ]:
## INSTALL langdetect first
# !pip install langdetect

In [ ]:
from langdetect import detect

In [ ]:
df_data_reviews['comments_langdect'] = df_data_reviews['comments'].apply(detect)

In [ ]:
df_data_reviews.head()

> #### Filter comment non English

In [ ]:
df_data_reviews[df_data_reviews['comments_langdect'] != 'en'].head() 

# it is not store into variable, so it is just works on this cell only

In [ ]:
df_data_reviews_non_eng = df_data_reviews[df_data_reviews['comments_langdect'] != 'en']

In [ ]:
df_data_reviews_non_eng.head()

#### I want do somes tranformation on this CALENDARS DATA

In [ ]:
query_calendars = """select * from calendars"""

In [ ]:
df_data_calendars = pd.read_sql(query_calendars, conn)

In [ ]:
df_data_calendars.head()

In [ ]:
df_data_calendars.isnull().sum()

> #### Remove null value from records

In [ ]:
df_data_calendars.dropna()

# it is not store into variable, so it is just works on this cell only

In [ ]:
df_data_calendars_clean = df_data_calendars.dropna()

In [ ]:
df_data_calendars_clean.isnull().sum()

#### 3. LOAD

Process that insert new data into storage ( we call it here DWH )

``` Why we use sqlachemy not pymysql ? Pandas require sqlachemy to connect with database, so we need it ```

- Here documentation from pandas

Using SQLAlchemy makes it possible to use any DB supported by that library. Legacy support is provided for sqlite3.Connection objects. The user is responsible for engine disposal and connection closure for the SQLAlchemy connectable See here**

In [ ]:
conn_dwh = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                               format(credential_db.db_jatimcamp5_DWH_username, 
                                                      credential_db.db_jatimcamp5_DWH_password, 
                                                      credential_db.db_jatimcamp5_DWH_host, 
                                                      credential_db.db_jatimcamp5_DWH_name))

conn_dwh

#### push clean calender into Datawarehouse (DWH) to provide data team 

In [ ]:
df_data_calendars_clean.to_sql(con=conn_dwh, name='calendars', if_exists='replace')

#### How to check the data ? we can use IDE database ( such as dbeaver, navicat, etc ) 

#### OR we can query from here

In [ ]:
query_cal = 'select * from calendars limit 10'

df_cal_dwh = pd.read_sql_query(query_cal, conn_dwh)

In [ ]:
df_cal_dwh.isnull().sum()

In [ ]:
df_cal_dwh.head()

#### ```Why do we load new data into new database ( jatimcamp5_dwh ) not in jatimcamp5 ?```